### 08. Pytorch Basics

This pytorch basics tutorial contains more examples for autograd, loading data, input pipline, pretrained model, save and load model.

#### Table of Contents

- [1. Basic autograd example 1](#heading)
- [2. Basic autograd example 2](#heading)
- [3. Loading data from numpy](#heading)
- [4. Input pipline](#heading)
- [5. Input pipline for custom dataset](#heading)
- [6. Pretrained model](#heading)
- [7. Save and load model](#heading)


#### 1. Basic autograd example 1

In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [2]:
# create tensors
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# build a computational graph
# y = 2 * x + 3
y = w * x + b

# compute gradients
y.backward()

# print out the gradients
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


#### 2. Basic autograd example 2

In [3]:
# create tensors of shape (10, 3) and (10, 2)
x = torch.randn(10, 3)
y = torch.randn(10, 2)
print(f'x: {x}')
print(f'y: {y}')

x: tensor([[-0.9527, -0.2505, -1.8054],
        [ 0.4457,  1.3306, -0.1702],
        [ 0.5204, -0.4415, -0.4162],
        [ 0.3132,  1.0229,  0.6401],
        [-0.3796, -0.3896,  1.2053],
        [-0.7884, -1.3956,  1.9586],
        [ 0.4727, -1.1065,  1.0103],
        [-0.3476, -1.1874, -1.2094],
        [-0.3993,  0.2940, -0.2490],
        [-1.3199, -0.4339, -0.8404]])
y: tensor([[-0.4410, -0.6859],
        [ 3.3284, -0.8028],
        [ 0.4173, -0.1413],
        [-0.3205,  0.3768],
        [ 0.7461,  0.5453],
        [ 0.9099,  0.6813],
        [-0.2392,  0.0340],
        [-0.1058,  0.1387],
        [-1.4546, -0.6930],
        [-0.0121,  0.9345]])
